# Festival Playlists

In [1]:
import os
import numpy as np
import pandas as pd
import requests
import json
import spotipy
from IPython.display import display

1. Use the Songkick API to get all the bands playing the festival
2. Use the Setlist.FM API to get the setlists
3. Use the Spotify API to create the playlists and add all the songs

### Set API credentials

In [2]:
setlistfm_api_key = os.getenv('SETLISTFM_API_KEY')
spotify_client_id = os.getenv('SPOTIFY_CLIENT_ID')
spotify_client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

### Setlist FM

#### Plan of action
1. Given a lineup (list of band names), get their Musicbrainz identifiers (`mbid`) via `https://api.setlist.fm/rest/1.0/search/artists`
2. Retrieve the setlists for each artist using their `mbid` via `https://api.setlist.fm/rest/1.0/artist/{artist_mbid}/setlists
`

In [15]:
lineup = pd.read_csv(
    '/Users/adrialuz/Desktop/repos/festival-playlists/data/PS2023.csv', header=None, names=['band'], encoding="ISO-8859-1"
)['band'].values

In [16]:
len(lineup)

177

In [17]:
lineup

array(['Pet Shop Boys', 'Confidence Man', 'Jake Bugg', 'La Paloma',
       'Blur', 'New Order', 'Halsey', 'Darkside', 'Ghost', 'Le Tigre',
       'NxWorries', 'Turnstile', 'Central Cee', 'Drain Gang',
       'Loyle Carner', 'Pusha T', 'Rema', 'Alex G', 'Amenra',
       'Arthur Verocai', 'Black Country New Road', 'Boris',
       'Built To Spill', 'DJ Playero', 'Emeralds', 'Folamour',
       'Hudson Mohawke', 'Machine Girl', 'Off', 'Perfume',
       'Pinkpantheress', 'Red Velvet', 'Self Esteem', 'Sudan Archives',
       'The Comet Is Coming', 'Yard Act', 'Antonim', 'Ascendant Vierge',
       'Blackhaine', 'Brutalismus 3000', 'Cabiria', 'Chica Gang', 'Come',
       'Flowerovlove', 'Heather', 'Heinali', 'Isabella Lovestory',
       'Jana Rush', 'Joe Crepusculo', 'Joe Unknown', 'Julia Colom',
       'Juliana Huxtable', 'Jasss', 'Karenn', 'Maral', 'Rhyw',
       'Salamanda', 'Shannen', 'Joe Cotch', 'Slauson Malone 1',
       'Terno Rei', 'Verraco', 'Kendrick Lamar', 'Depeche Mode',
       'B

In [18]:
artists_url = 'https://api.setlist.fm/rest/1.0/search/artists'

In [19]:
lineup_mbids = []
not_found = []

for name in lineup:
    req = requests.get(artists_url,
                       headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                       params={'artistName': name, 'p': 1, 'sort': 'relevance'}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(artists_url,
                           headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                           params={'artistName': name, 'p': 1, 'sort': 'relevance'}
        )
        i += 1
    
    if req.ok:
        artist_response = req.json()['artist']
        num_artists = len(artist_response)
        if num_artists > 1:
            for i in range(num_artists):
                if artist_response[i]['name'].lower() == name.lower():
                    mbid = artist_response[i]['mbid']
                    lineup_mbids.append({'name': name, 'mbid': mbid})
                    break
        elif num_artists == 1:
            mbid = artist_response[0]['mbid']
            lineup_mbids.append({'name': name, 'mbid': mbid})
        elif num_artists == 0:
            print(f'No results I think for {name}')
        else:
            print(f'WTF {name}?')
                    
    else:
        print(f'Couldn\'t find {name}')
        not_found.append(name)

Couldn't find Antonim
Couldn't find Chica Gang
Couldn't find Julia Colom
Couldn't find Slauson Malone 1
Couldn't find Avalon Emerson the Charm
Couldn't find Low Jack x Lala Ce
Couldn't find NiÃ±a Coyote ETA Chico Tornado
Couldn't find Tongue in the Mind
Couldn't find James Ellis Ford
Couldn't find Melina Serser
Couldn't find The Drift Institute
Couldn't find Wooden Wisdom


In [20]:
lineup_mbids

[{'name': 'Pet Shop Boys', 'mbid': 'be540c02-7898-4b79-9acc-c8122c7d9e83'},
 {'name': 'Confidence Man', 'mbid': '9fab430a-f1af-4a69-bf68-6857a7a50a68'},
 {'name': 'Jake Bugg', 'mbid': '0d9905e6-add6-4713-94ca-9f686f9b5e9b'},
 {'name': 'La Paloma', 'mbid': 'a00f6bad-bbd6-463f-96ed-6f920ebbdd5c'},
 {'name': 'Blur', 'mbid': 'ba853904-ae25-4ebb-89d6-c44cfbd71bd2'},
 {'name': 'New Order', 'mbid': 'f1106b17-dcbb-45f6-b938-199ccfab50cc'},
 {'name': 'Halsey', 'mbid': '3377f3bb-60fc-4403-aea9-7e800612e060'},
 {'name': 'Darkside', 'mbid': '637684f9-8483-49a7-ac85-517a76e0344c'},
 {'name': 'Ghost', 'mbid': '2bcf2e02-5bc3-4c76-bf76-41126cb11444'},
 {'name': 'Le Tigre', 'mbid': '2d67239c-aa40-4ad5-a807-9052b66857a6'},
 {'name': 'NxWorries', 'mbid': '96c5c9a3-6c9b-41a7-90cc-e7375afbfde1'},
 {'name': 'Turnstile', 'mbid': '7b748dac-f5ce-45a7-9b95-c1d8b5b013ed'},
 {'name': 'Central Cee', 'mbid': 'b0337af1-8d93-4671-b6c9-ba306bf942bf'},
 {'name': 'Drain Gang', 'mbid': 'ad9114ca-c228-4f4c-bd86-84020dcf5a

In [21]:
not_found

['Antonim',
 'Chica Gang',
 'Julia Colom',
 'Slauson Malone 1',
 'Avalon Emerson the Charm',
 'Low Jack x Lala Ce',
 'NiÃ±a Coyote ETA Chico Tornado',
 'Tongue in the Mind',
 'James Ellis Ford',
 'Melina Serser',
 'The Drift Institute',
 'Wooden Wisdom']

In [22]:
artist_setlist = []

for a in lineup_mbids:
    songs_played = []
    mbid = a['mbid']
    setlists_url = f'https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists'
    
    req = requests.get(setlists_url,
             headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
             params={'p': 1}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(setlists_url,
                 headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                 params={'p': 1}
        )
        i += 1
            
    if req.ok:
    
        setlist_response = req.json()['setlist']
        num_setlists = len(setlist_response)

        for i in range(num_setlists):
            setlist = setlist_response[i]['sets']['set']
            num_sections = len(setlist)
            total_songs = []
            for p in range(num_sections):
                total_songs += setlist[p]['song']
            num_songs = len(total_songs)

            for i in range(num_songs):
                song = total_songs[i]
                song_title = song['name']
                # if the song is a cover add the original artist to the song title
                if 'cover' in song:
                    song_title += ' {}'.format(song['cover']['name'])
                songs_played.append(song_title)
                                           
        most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)

        artist_setlist.append({
            'artist': a['name'],
            'setlist': most_played_songs
        })
    else:
        not_found.append(a['name'])

/var/folders/5g/z2_s3_7d75qgyjp72n_8q0p40000gq/T/ipykernel_55352/1942038417.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)


In [23]:
not_found

['Antonim',
 'Chica Gang',
 'Julia Colom',
 'Slauson Malone 1',
 'Avalon Emerson the Charm',
 'Low Jack x Lala Ce',
 'NiÃ±a Coyote ETA Chico Tornado',
 'Tongue in the Mind',
 'James Ellis Ford',
 'Melina Serser',
 'The Drift Institute',
 'Wooden Wisdom',
 'La Paloma',
 'Rhyw',
 'Joe Cotch',
 'Israel Fernandez y Diego del Morao',
 'Carlota Flaneur',
 'Honour',
 'Los Hacheros',
 'Maddy Maia',
 'Nazire',
 'Velmondo',
 'Rosalia']

In [24]:
artist_setlist[:5]

[{'artist': 'Pet Shop Boys',
  'setlist': ['Suburbia',
   'You Were Always on My Mind Gwen McCrae',
   'Being Boring',
   'West End Girls',
   "It's a Sin",
   'Vocal',
   'Can You Forgive Her?',
   'Heart',
   'Dreamland',
   "It's Alright Sterling Void",
   "Opportunities (Let's Make Lots of Money)",
   'So Hard',
   'Love Comes Quickly',
   "Where the Streets Have No Name (I Can't Take My Eyes Off You)",
   "I Don't Know What You Want but I Can't Give It Any More"]},
 {'artist': 'Confidence Man',
  'setlist': ['Toy Boy',
   'Luvin U Is Easy',
   'Holiday',
   'Boyfriend (Repeat)',
   "Don't You Know I'm in a Band",
   'Relieve the Pressure',
   'Out the Window',
   'Trumpet Song',
   'C.O.O.L Party',
   'Feels Like a Different Thing',
   'Flute Song',
   'Break It Bought It',
   'Does It Make You Feel Good?',
   'What I Like',
   'Woman']},
 {'artist': 'Jake Bugg',
  'setlist': ['Lightning Bolt',
   'All I Need',
   'Two Fingers',
   'Seen It All',
   'Simple as This',
   'Slumville

In [25]:
setlist_lengths = []
short_or_empty_setlist = []

for i in range(len(artist_setlist)):
    n_songs = len(artist_setlist[i]['setlist'])
    setlist_lengths.append({
        'artist': artist_setlist[i]['artist'],
        'n_songs': n_songs
    })
    
    if n_songs < 5:
        short_or_empty_setlist.append(artist_setlist[i]['artist'])

In [26]:
len(short_or_empty_setlist)

43

In [27]:
short_or_empty_setlist

['NxWorries',
 'DJ Playero',
 'Emeralds',
 'Folamour',
 'Hudson Mohawke',
 'Blackhaine',
 'Brutalismus 3000',
 'Flowerovlove',
 'Heinali',
 'Isabella Lovestory',
 'Jana Rush',
 'Joe Unknown',
 'Juliana Huxtable',
 'Jasss',
 'Karenn',
 'Maral',
 'Salamanda',
 'Verraco',
 'Beak',
 'Daphni',
 'John Talabot',
 'TSHA',
 'VTSS',
 'LSDXOXO',
 'Bill Kouligas',
 'Gigi FM',
 'Tottie',
 'The Soft Pink Truth',
 'Upsammy',
 'Charlotte de Witte',
 'Eddie Palmieri',
 'Jayda G',
 'Nia Archives',
 'Two Shell',
 'Anish Kumar',
 'CCL',
 'DJ Storm',
 'Josey Rebelle',
 'Mala',
 'Om Unit',
 'Pional',
 'Ubaldo',
 'DJ Fitz']

### Spotify

In [28]:
username = 'adrialuz'
scope = 'playlist-modify-public'

token = spotipy.util.prompt_for_user_token(username, scope, redirect_uri='http://localhost:9090')

sp = spotipy.Spotify(auth=token)

sp.trace = False

In [29]:
spotify_ids = []
for a in short_or_empty_setlist:
    search_result = sp.search(
        f'artist:{a}', limit=5, type='artist', market='GB'
    )['artists']['items']
    
    if search_result:
        for i in range(len(search_result)):
            name = search_result[i]['name']
            if name.lower() == a.lower():
                artist_id = search_result[i]['id']
                spotify_ids.append(artist_id)
                break
            else:
                pass
    else:
        print(f'Couldn\'t find {a} on Spotify.')

In [30]:
spotify_ids[:5]

['6PEMFpe3PTOksdV4ZXUpbE',
 '7HZ3dOfMWYvts1LC9fJq44',
 '45btnwhUWFlR7Op5oTfDPv',
 '6pJY5At9SiMpAOBrw9YosS',
 '6olWbKW2VLhFCHfOi0iEDb']

In [31]:
sp.artist('59xdAObFYuaKO2phzzz07H')['name']

'Special Request'

In [32]:
popular_songs = []

for artist_id in spotify_ids:
    search_results = sp.artist_top_tracks(artist_id, country='GB')['tracks']

    top_songs = []
    if search_results:
        for i in range(len(search_results)):
            song_name = search_results[i]['name']
            top_songs.append(song_name)
        popular_songs.append({
            'artist': sp.artist(artist_id)['name'],
            'setlist': top_songs
        })
    else:
        print(artist_id, sp.artist(artist_id)['name'])

7HZ3dOfMWYvts1LC9fJq44 DJ Playero


In [33]:
popular_songs[:5]

[{'artist': 'NxWorries',
  'setlist': ['Where I Go (feat. H.E.R.)',
   'Suede',
   'itkanbe[sonice]',
   'What More Can I Say',
   'Lyk Dis',
   'Another Time',
   'Wngs',
   'Scared Money',
   'Get Bigger / Do U Luv',
   'Droogs']},
 {'artist': 'Emeralds',
  'setlist': ['Goes By',
   'Candy Shoppe',
   'Magic',
   'The Cycle Of Abuse',
   'Nereus (Spirits Over the Lake)',
   'Now You See Me',
   'Photosphere',
   'Double Helix',
   'Genetic',
   'The Quaking Mess']},
 {'artist': 'Folamour',
  'setlist': ['Freedom',
   'These Are Just Places To Me Now',
   'Ya Just Need 2 Believe in Yaself',
   'Petit Prince Du Macadam',
   'When U Came into My Life',
   'Friends (feat. Tim Ayre)',
   'I Miss Having Someone To Talk To',
   'Alive',
   'After Winter Must Come Spring feat. Elbi',
   'Devoted To U']},
 {'artist': 'Hudson Mohawke',
  'setlist': ['Cbat',
   'VSOD - Velvet Sky of Dreams',
   'Chimes',
   'Helium - Hudson Mohawke Remix',
   'Is It Supposed',
   'Bicstan',
   'Watch Dogs Theme

Get the URI codes for each track

In [35]:
uris = []
missing_songs = []

for a in (artist_setlist + popular_songs):
    artist = a['artist']
    setlist = a['setlist']
    for s in setlist:
        s = s.replace(',', '').replace('\'', '').replace('"', '').replace('.', '').replace(
            '?', '').replace(')', '').replace('(', '').replace('/', '').replace(
            '\\', '').replace('&', '').replace('-', '')
        items = sp.search(q=f'artist:{artist} track:{s}', limit=1)['tracks']['items']
        if items:
            uri = items[0]['id']
            uris.append(uri)
        else:
            items = sp.search(q=f'track:{s}', limit=1)['tracks']['items']
            if items:
                if items != [None]:
                    uri = items[0]['id']
                    uris.append(uri)
            else:
                missing_songs.append({
                    'artist': artist,
                    'song': s
                })

In [36]:
len(uris)

1837

In [37]:
len(missing_songs)

43

In [38]:
missing_songs

[{'artist': 'Halsey', 'song': '1121  Die for Me'},
 {'artist': 'Darkside', 'song': 'The Only Shrine Ive Ever Seen'},
 {'artist': 'Turnstile', 'song': 'TLC TURNSTILE LOVE CONNECTION'},
 {'artist': 'Drain Gang', 'song': 'Girls Just Wanna Have Fun Bladee  ECCO2K'},
 {'artist': 'Amenra', 'song': 'Thurifer et Clamor ad te Veniat'},
 {'artist': 'Black Country New Road', 'song': 'TurbinesPigs'},
 {'artist': 'Boris', 'song': 'Fundamental Error Gudon'},
 {'artist': 'Boris', 'song': 'Kiki no Ue'},
 {'artist': 'Boris', 'song': 'HxCxHxC Perforation Line'},
 {'artist': 'Machine Girl', 'song': 'Global Fandemic Track 2 Because You Can'},
 {'artist': 'Perfume', 'song': 'Mawaru Kagami'},
 {'artist': 'Perfume', 'song': 'Sayonara Plastic World'},
 {'artist': 'Perfume', 'song': 'Hatenabito'},
 {'artist': 'The Comet Is Coming', 'song': 'Shabaka Hutchings Sax Solo'},
 {'artist': 'Cabiria', 'song': 'DISCOCAFÉ'},
 {'artist': 'Karenn', 'song': 'Live Set  Wall of Sound ADE 2017'},
 {'artist': 'The Moldy Peaches

In [39]:
divisor = int(np.floor(len(uris) / np.ceil(len(uris) / 100)))
times = int(np.floor(len(uris) / divisor))

In [41]:
for i in range(times):
    subset = uris[divisor*i:divisor*(i+1)]
    sp.user_playlist_add_tracks(username, playlist_id='2rNEeCnxYeMCwLXSFCQvzH',
                                tracks=subset)